**Avant de lancer ce notebook merci de générer les csv à l'aide du [notebook suivant](Create-csv-files.ipynb).**

Dénormalisation retenue:

In [1]:
{
    "members":
    {
        "member_no": "1",
        "lastname": "DOUGLAS",
        "firstname": "Michael",
        "phone_number":"blabla",

        "location":
        {
            "country":"France",
            "region_no":"IDF",
            "city":"Paris",
            "mail_code":"75000",
            "street":"blabla"
        },

        "balance":
        {
            "prev_balance": "1000",
            "curr_balance": "1200"
        },

        "_comment":"No need to have a list, each employee have only 1 corporation",
        "corporation":
        {
            "corp_no":1,
            "corp_name":"blabla",
            "region_no":"XX"
        },

        "charges": [
            {
                "charge_no": "125",
                "category_no": "12",
                "category_desc": "Travel",
                "charge_dt": "2000-02-22 10:43:38",
                "charge_amount": "336.0000",

                "provider_no":12,
                "provider_name":"BNP Paribas",
                "provider_phone_number": "blabla",
                "payments":[1546, 1547,... , 1578]
            }
        ]
    },
     "providers":
        {
            "provider_no":1,
            "provider_name":"BNP",
            "location":
            {
                "country":"France",
                "region_no":"IDF",
                "city":"Paris",
                "mail_code":"75000"
            }
        },
 
 
    "payments":
        {
            "payment_no":1546,
            "payment_dt":"2000-02-22 10:43:38",
            "payment_amt":125.00
        }
}


{'members': {'member_no': '1',
  'lastname': 'DOUGLAS',
  'firstname': 'Michael',
  'phone_number': 'blabla',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000',
   'street': 'blabla'},
  'balance': {'prev_balance': '1000', 'curr_balance': '1200'},
  '_comment': 'No need to have a list, each employee have only 1 corporation',
  'corporation': {'corp_no': 1, 'corp_name': 'blabla', 'region_no': 'XX'},
  'charges': [{'charge_no': '125',
    'category_no': '12',
    'category_desc': 'Travel',
    'charge_dt': '2000-02-22 10:43:38',
    'charge_amount': '336.0000',
    'provider_no': 12,
    'provider_name': 'BNP Paribas',
    'provider_phone_number': 'blabla',
    'payments': [1546, 1547, Ellipsis, 1578]}]},
 'providers': {'provider_no': 1,
  'provider_name': 'BNP',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000'}},
 'payments': {'payment_no': 1546,
  'payment_dt': '2000-02-22 10:43:3

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
Members = pd.read_csv('Members.csv')
Corporations = pd.read_csv('Corporations.csv')
Categories = pd.read_csv('Categories.csv')
Providers = pd.read_csv('Prodivers.csv')
Charges = pd.read_csv('Charges.csv')
Statements = pd.read_csv('Statements.csv')
Payments = pd.read_csv('Payments.csv')

In [4]:
Charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
0,0,37069,36,2021-12-23,907,1,0
1,2,6166,36,2020-09-05,908,0,1
2,2,5000,60,2021-01-26,96,12,2
3,0,321,60,2020-06-07,868,14,3
4,2,5582,36,2020-08-08,548,1,4
...,...,...,...,...,...,...,...
495,2,25916,24,2021-06-20,835,5,495
496,1,15064,60,2020-11-18,487,14,496
497,0,5000,24,2020-12-28,640,10,497
498,2,5000,48,2021-09-19,544,15,498


In [5]:
Payments

,payment_amt,due_dt,charge_no,member_no,payment_no
0,5.350000,2020-07-07,3,868,132
1,5.350000,2020-08-07,3,868,133
2,208.333333,2020-08-30,6,128,264
3,846.944444,2020-03-29,7,956,288
4,846.944444,2020-04-29,7,956,289
...,...,...,...,...,...
547,372.916667,2020-04-07,489,403,18146
548,372.916667,2020-05-07,489,403,18147
549,372.916667,2020-06-07,489,403,18148
550,372.916667,2020-07-07,489,403,18149


In [6]:
member_no = 907

charges = Charges[Charges.member_no == member_no]
charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
0,0,37069,36,2021-12-23,907,1,0


In [7]:
providers_no = charges.provider_no.values
providers_no

array([1])

In [8]:
providers = []
for provider_no in providers_no:
    provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
    providers.append(provider)
providers

[provider_no                    1
 provider_name          Citigroup
 phone_number          2000140380
 country            United States
 region_no                     AK
 city                   Anchorage
 mail_code                  99507
 street           6160 Norm Drive
 Name: 1, dtype: object]

In [9]:
charges_no = charges.charge_no.values
charges_no

array([0])

In [10]:
payments = []
for charge_no in charges_no:
    payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
    payments.append(list(payments_no))
payments

[[]]

In [11]:
charges_json = []
for i in range(len(charges_no)):
    charge = charges.iloc[i-1]
    provider = providers[i]
    payments_list = payments[i]
    charge_json = {
        "charge_no": charge.charge_no,
        "category_no": charge['Charge type'],
        #"category_desc": "Travel",
        "charge_dt": charge['dt'],
        "charge_amount": charge['Charge amount'],

        "provider_no": provider.provider_no,
        "provider_name": provider.provider_name,
        "provider_phone_number": provider.phone_number,
        "payments": payments_list
    }
    charges_json.append(charge_json)
    
charges_json

[{'charge_no': 0,
  'category_no': 0,
  'charge_dt': '2021-12-23',
  'charge_amount': 37069,
  'provider_no': 1,
  'provider_name': 'Citigroup',
  'provider_phone_number': 2000140380,
  'payments': []}]

In [12]:
member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
member

member_no                    907
lastname                    Boyd
firstname                   John
phone_number          8914287908
country            United States
region_no                     CA
city                     Oakland
mail_code                  94605
street          2747 68th Avenue
corp_no                        7
Name: 907, dtype: object

In [13]:
corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
corporation

corp_no                            7
corp_name       Spencer, Lee and Cox
phone_number              2491025911
country                United States
region_no                         CA
city                         Fremont
mail_code                      94536
street              3526 Sutton Loop
Name: 7, dtype: object

In [14]:
member_json = {
    "member_no": member_no,
    "lastname": member.lastname,
    "firstname": member.firstname,
    "phone_number": member.phone_number,

    "location":
    {
        "country": member.country,
        "region_no": member.region_no,
        "city": member.city,
        "mail_code": member.mail_code,
        "street": member.street
    },

    #"balance":
    #{
        #"prev_balance": "1000",
        #"curr_balance": "1200"
    #},
    "corporation":
    {
        "corp_no": corporation.corp_no,
        "corp_name": corporation.corp_name,
        "region_no": corporation.region_no
    },

    "charges": charges_json
}
member_json

{'member_no': 907,
 'lastname': 'Boyd',
 'firstname': 'John',
 'phone_number': 8914287908,
 'location': {'country': 'United States',
  'region_no': 'CA',
  'city': 'Oakland',
  'mail_code': 94605,
  'street': '2747 68th Avenue'},
 'corporation': {'corp_no': 7,
  'corp_name': 'Spencer, Lee and Cox',
  'region_no': 'CA'},
 'charges': [{'charge_no': 0,
   'category_no': 0,
   'charge_dt': '2021-12-23',
   'charge_amount': 37069,
   'provider_no': 1,
   'provider_name': 'Citigroup',
   'provider_phone_number': 2000140380,
   'payments': []}]}

In [15]:
def get_charges(member_no):
    charges = Charges[Charges.member_no == member_no]
    return charges

def get_member(member_no):
    member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
    return member

def get_providers(charges):
    providers_no = charges.provider_no.values
    providers = []
    for provider_no in providers_no:
        provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
        providers.append(provider)
    return providers

def get_payments(charges):
    charges_no = charges.charge_no.values
    payments = []
    for charge_no in charges_no:
        payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
        payments.append(list(payments_no))
    return payments

def get_charges_json(member_no):
    charges = get_charges(member_no)
    providers = get_providers(charges)
    payments = get_payments(charges)
    
    charges_json = []
    for i in range(len(charges_no)):
        charge = charges.iloc[i-1]
        provider = providers[i]
        payments_list = payments[i]
        json = {
            "charge_no": charge.charge_no,
            "category_no": charge['Charge type'],
            #"category_desc": "Travel",
            "charge_dt": charge['dt'],
            "charge_amount": charge['Charge amount'],

            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "provider_phone_number": provider.phone_number,
            "payments": payments_list
        }
        charges_json.append(json)

    return charges_json

def get_corporation(member):
    corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
    return corporation

def get_member_json(member_no):
    charges_json = get_charges_json(member_no)
    member = get_member(member_no)
    corporation = get_corporation(member)
    
    member_json = {
        "member_no": member_no,
        "lastname": member.lastname,
        "firstname": member.firstname,
        "phone_number": member.phone_number,

        "location":
        {
            "country": member.country,
            "region_no": member.region_no,
            "city": member.city,
            "mail_code": member.mail_code,
            "street": member.street
        },

        #"balance":
        #{
            #"prev_balance": "1000",
            #"curr_balance": "1200"
        #},
        "corporation":
        {
            "corp_no": corporation.corp_no,
            "corp_name": corporation.corp_name,
            "region_no": corporation.region_no
        },

        "charges": charges_json
    }
    return member_json

def get_members_json():
    return pd.Series(Charges.member_no.unique()).apply(lambda member_no: get_member_json(member_no)).to_list()
    # On utilise .uniques() pour éviter les doublons

In [16]:
get_members_json()

[{'member_no': 907,
  'lastname': 'Boyd',
  'firstname': 'John',
  'phone_number': 8914287908,
  'location': {'country': 'United States',
   'region_no': 'CA',
   'city': 'Oakland',
   'mail_code': 94605,
   'street': '2747 68th Avenue'},
  'corporation': {'corp_no': 7,
   'corp_name': 'Spencer, Lee and Cox',
   'region_no': 'CA'},
  'charges': [{'charge_no': 0,
    'category_no': 0,
    'charge_dt': '2021-12-23',
    'charge_amount': 37069,
    'provider_no': 1,
    'provider_name': 'Citigroup',
    'provider_phone_number': 2000140380,
    'payments': []}]},
 {'member_no': 908,
  'lastname': 'Garcia',
  'firstname': 'Laura',
  'phone_number': 7471013762,
  'location': {'country': 'United States',
   'region_no': 'AR',
   'city': 'Farmington',
   'mail_code': 72730,
   'street': '110 Allen Place'},
  'corporation': {'corp_no': 48,
   'corp_name': 'Jones Group',
   'region_no': 'AK'},
  'charges': [{'charge_no': 216,
    'category_no': 2,
    'charge_dt': '2021-12-03',
    'charge_amoun

In [17]:
def get_providers_json():
    providers = [
        {
            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "location":
            {
                "country": provider.country,
                "region_no": provider.region_no,
                "city": provider.city,
                "mail_code": provider.mail_code
            }
        } for (_,provider) in Providers.iterrows()]
    return providers

def get_payments_json():
    payments = [{
            "payment_no": payment.payment_no,
            "payment_dt": payment.due_dt,
            "payment_amt": payment.payment_amt
        } for (_,payment) in Payments.iterrows()]
    return payments

def final_json():
    return {
        'members': get_members_json(),
        'providers': get_providers_json(),
        'payments': get_payments_json()
    }

# Class pour résoudre les problèmes d'encoding entre numpy et le module json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def export_final_json():
    data = final_json()
    with open('final-data.json', 'w') as outfile:
        json.dump(data, outfile, cls=NpEncoder, indent=4)

In [18]:
export_final_json()